In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from itertools import product

data = {}
PATH = '/kaggle/input/inseedeathsfrance/'
output = './'
                         
for dirname, _, filenames in os.walk(PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if filename.endswith('.csv'):
            data[filename] = pd.read_csv(os.path.join(dirname, filename), sep=';', encoding='latin')
    print('-' * 100)

In [ ]:
ages = ['0_24ans', '25_49ans', '50_64ans', '65_74ans', '75_84ans', '85ans_plus']
sexes = ['Femmes', 'Hommes']
annees = [2018, 2019, 2020]
series = {}
index_to_drop = [31+29-1, 366+31+29-1]

for file, df in data.items():
    
    cols = df.columns
    
#     if 'deces_parage' in file:
#         print(file)
#         for age in ages:
#             print(age)
#             print(df[list(filter(lambda c: age in c, cols))].sum())
            
    if "age_jour" in file:
#         print(file)
        df = df[list(filter(lambda c: 'Deces' in c, cols))]
        row_0 = df.loc[0]
        df = df.diff()
        df.loc[0] = row_0
        for age in ages:
            for sex in sexes:
                series[f'{sex}_{age}'] = pd.concat([df[f'{sex}_{age}_Deces{y}'] for y in annees]).reset_index(drop=True).drop(index_to_drop).reset_index(drop=True).astype(int)
    
#     print('-' * 100)

In [ ]:
from datetime import date, timedelta

sdate = date(2018, 1, 1)   # start date
edate = date(2020, 12, 31)   # end date

new_index = [sdate + timedelta(days=i) for i in range((edate - sdate).days + 1)]

for i, s in series.items():
    s.index = new_index
    series[i] = s

In [ ]:
for age in ages:
    series[age] = pd.concat([series[f'{sex}_{age}'] for sex in sexes], axis=1)
    series[age].columns = sexes
    series[age].rolling(window=7, min_periods=1, center=True).mean().plot(figsize=(20, 5))
    plt.title(age)
    plt.show()

In [ ]:
for sex in sexes:
    series[sex] = pd.concat([series[f'{sex}_{age}'] for age in ages], axis=1)
    series[sex].columns = ages
    series[sex].rolling(window=7, min_periods=1, center=True).mean().plot(figsize=(20, 5))
    plt.title(sex)
    plt.show()

In [ ]:
for s, a in product(sexes, ages):
    col = f'{s}_{a}'
    t = series[col]
    fig = plt.figure(figsize=(20, 5))
    labels = []
    for y, test in t.groupby(pd.to_datetime(t.index, format='%Y-%m-%d').year):
        labels.append(y)
        test.index = [s.strftime('%B-%d') for s in test.index]
        test.rolling(window=7, min_periods=1, center=True).mean().plot(fig=fig)
    plt.legend(labels)
    plt.title(col)

In [ ]:
for a in ages:
    t = series[a].sum(axis=1)
    fig = plt.figure(figsize=(20, 5))
    labels = []
    for y, test in t.groupby(pd.to_datetime(t.index, format='%Y-%m-%d').year):
        labels.append(y)
        test.index = [s.strftime('%B-%d') for s in test.index]
        test.rolling(window=7, min_periods=1, center=True).mean().plot(fig=fig)
    plt.legend(labels)
    plt.title(a)